In [4]:
import sys, os
sys.path.append(os.path.expanduser('~') + '/repos/pyQSYS/build/qsys')
from pyQSYS import *
import scipy.sparse.linalg

In [5]:
model = RealSpaceFermions(clusterName="square_8d2")
[model.momentum[i].k for i in range(len(model.momentum))]

[[0.0, 0.0],
 [-3.141592653589793, 0.0],
 [-1.5707963267948966, 1.5707963267948966],
 [0.0, 3.141592653589793],
 [-4.71238898038469, 1.5707963267948966],
 [-3.141592653589793, 3.141592653589793],
 [-1.5707963267948966, 4.71238898038469],
 [-4.71238898038469, 4.71238898038469]]

In [6]:
model = SpinHalf(clusterName = "square_16q0")
#model = SpinHalf(clusterName = "square_4d0")
H_Op = Operator()
statemap = model.createStateMap()

h_field = 0.6
J = 1

for site in model.sites:
    H_Op += 2 * h_field * Sz_(site)

for site in model.sites:
    H_Op += 4 * J * (SdotS_(site, site + [1,0]) + SdotS_(site, site + [0,1]))

Ham = Matrix('double', H_Op, model, statemap)


In [9]:
Ham_CSR = CSRMatrix('cdouble', H_Op, model, statemap)

In [12]:
Ham_CSR.conjugate().T

<65536x65536 sparse matrix of type '<class 'numpy.complex128'>'
	with 1109756 stored elements in Compressed Sparse Column format>

In [8]:
help(Ham)

Help on SparseMatrix in module pyQSYS object:

class SparseMatrix(pybind11_builtins.pybind11_object)
 |  Efficient sparse-matrix interface
 |  
 |  Method resolution order:
 |      SparseMatrix
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(...)
 |      __repr__(self: pyQSYS.SparseMatrix) -> str
 |  
 |  __str__(...)
 |      __str__(self: pyQSYS.SparseMatrix) -> str
 |  
 |  dot(...)
 |      dot(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. dot(self: pyQSYS.SparseMatrix, arg0: numpy.ndarray[numpy.float64[m, 1]]) -> Union[numpy.ndarray[numpy.float64[m, 1]], numpy.ndarray[numpy.complex128[m, 1]]]
 |      
 |      Calculate a matrix-vector product.
 |      
 |      2. dot(self: pyQSYS.SparseMatrix, arg0: numpy.ndarray[numpy.complex128[m, 1]]) -> numpy.ndarray[numpy.complex128[m, 1]]
 |    

In [42]:
evals, evecs = scipy.sparse.linalg.eigsh(Ham, k = 1, which = "SA")
gs = evecs[:,0]
evals

array([-44.91393283])

In [45]:
Szs_Op = 2 *sum([Sz_(i) for i in model.sites])
Szcorr_Op =4 * sum([Sz_(i)*Sz_(i + [1,0])+Sz_(i)*Sz_(i + [0,1]) for (i) in model.sites])


Szs = CSRMatrix('double', Szs_Op, model, statemap)
Szcorr = CSRMatrix('double', Szcorr_Op, model, statemap)

print("The exact value of <Sz> is ", gs @ (Szs @ gs))
print("The exact value of <SzSz> is ", gs @ (Szcorr @ gs))

The exact value of <Sz> is  -6.420709171343364e-32
The exact value of <SzSz> is  -3.7428277361429503


In [46]:
3.7428277361429503*4

14.971310944571801

array([22.56])